In [13]:
# Setup Sets
markets = ["Facebook", "Instagram", "Twitter"]
buckets = [1, 2]

In [14]:
# Setup Data
clicks_per_dollar = {("Facebook", 1):0.10, ("Facebook", 2):0.15, \
                     ("Instagram", 1):0.12, ("Instagram", 2):0.18, \
                     ("Twitter", 1):0.11, ("Twitter", 2):0.19}
conversion_rate = {"Facebook":0.10, \
                   "Instagram":0.14, \
                   "Twitter":0.12 }
cost_breakpoint = {"Facebook":1000, \
                   "Instagram":1500, \
                   "Twitter":900 }
total_budget = 3000
big_m = 1000000

In [15]:
# Import the PuLP Library
from pulp import *

In [16]:
# Create a Problem object
problem = LpProblem("Marketing Spend", LpMaximize)

In [17]:
# Setup Decision Variables
spend = LpVariable.dicts("Spend", \
                         [(m, b) for m in markets for b in buckets], \
                         lowBound=0, upBound=None, cat=LpContinuous)
use_bucket_2 = LpVariable.dicts("Use_Bucket_2", markets, 0, 1, LpInteger)

In [18]:
# Set objective to maximize Customer Acquisition
problem += lpSum([spend[(m,b)]*clicks_per_dollar[(m,b)]*conversion_rate[m] \
                 for m in markets \
                 for b in buckets])

In [19]:
# Create constraint for total budget
problem += lpSum([spend[(m,b)] for m in markets for b in buckets]) <= total_budget, "Budget Constraint"

In [20]:
# If Bucket 1 is filled, can use Bucket 2
for m in markets:
    problem += spend[(m,1)] >= cost_breakpoint[m] * use_bucket_2[m], f"Bucket1Filled_{m}"

In [21]:
# Bucket 2 can be used if use_bucket_2 turned on
for m in markets:
    problem += spend[(m,2)] <= big_m * use_bucket_2[m], f"Bucket2Available_{m}"

In [22]:
problem.solve()

1

In [23]:
# Check the status
status = LpStatus[problem.status]
print("Status:", LpStatus[problem.status])

# It is best practice to always check the status of the solve before using the
# values of the decision variables
if status == "Optimal":
    for v in problem.variables():
        print(v.name, "=", v.varValue)
    print(f"Total Conversions = {value(problem.objective):n}")

Status: Optimal
Spend_('Facebook',_1) = 0.0
Spend_('Facebook',_2) = 0.0
Spend_('Instagram',_1) = 1500.0
Spend_('Instagram',_2) = 1500.0
Spend_('Twitter',_1) = 0.0
Spend_('Twitter',_2) = 0.0
Use_Bucket_2_Facebook = 0.0
Use_Bucket_2_Instagram = 1.0
Use_Bucket_2_Twitter = 0.0
Total Conversions = 63
